<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread google-auth-oauthlib pandas gradio

In [2]:
# 區塊 2: 授權並連接到 Google Sheets
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# 進行授權
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- 請填寫您的 Google Sheet 資訊 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1D6O9bWu97eWo0fls82iasAFiRQ75tYUk5Hi5Y6XVux8/edit?usp=sharing"
# ------------------------------------

try:
    # 開啟 Google Sheet 並選取第一個工作表
    worksheet = gc.open_by_url(SHEET_URL).sheet1
    print("✅ Google Sheet 連接成功！")
except gspread.exceptions.SpreadsheetNotFound:
    print("🛑 錯誤：找不到指定的 Google Sheet，請檢查連結是否正確。")
except Exception as e:
    print(f"🛑 發生未知錯誤：{e}")

✅ Google Sheet 連接成功！


In [3]:
# 區塊 3: 作業管理核心功能
import json
from datetime import datetime

# --- 欄位名稱設定 (請確保與您的 Sheet 欄位名稱一致) ---
COLUMNS = ["科目", "作業內容", "繳交日期", "狀態"]
# ----------------------------------------------------

def load_data_from_sheet():
    """從 Google Sheet 載入資料並轉換成 Pandas DataFrame"""
    try:
        data = worksheet.get_all_records()
        if not data: # 如果工作表是空的
            df = pd.DataFrame(columns=COLUMNS)
        else:
            df = pd.DataFrame(data)
            # 確保所有必要欄位都存在
            for col in COLUMNS:
                if col not in df.columns:
                    df[col] = ""
        # 轉換日期格式以便排序與查詢
        df['繳交日期'] = pd.to_datetime(df['繳交日期'], errors='coerce').dt.strftime('%Y-%m-%d')
        df = df.sort_values(by="繳交日期", ascending=True)
        return df
    except Exception as e:
        print(f"讀取資料時發生錯誤: {e}")
        return pd.DataFrame(columns=COLUMNS)

def save_data_to_sheet(df):
    """將 DataFrame 的資料回存到 Google Sheet"""
    try:
        # 清除整個工作表
        worksheet.clear()
        # 寫入欄位名稱與資料
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
        print("✅ 資料已成功同步至 Google Sheet！")
    except Exception as e:
        print(f"🛑 寫入資料時發生錯誤: {e}")


def add_task(subject, content, due_date, status):
    """新增一筆作業紀錄"""
    if not all([subject, content, due_date, status]):
        return "🛑 所有欄位都必須填寫！", load_data_from_sheet()
    try:
        # 格式化日期
        due_date_str = pd.to_datetime(due_date).strftime('%Y-%m-%d')
        new_task = pd.DataFrame([[subject, content, due_date_str, status]], columns=COLUMNS)

        df = load_data_from_sheet()
        df_updated = pd.concat([df, new_task], ignore_index=True)

        save_data_to_sheet(df_updated)
        return "✅ 任務新增成功！", df_updated
    except Exception as e:
        return f"🛑 新增失敗：{e}", load_data_from_sheet()


def delete_task(task_identifier):
    """刪除一筆指定的作業紀錄"""
    if not task_identifier:
        return "🛑 請選擇要刪除的任務！", load_data_from_sheet(), gr.Dropdown(choices=[])

    df = load_data_from_sheet()
    # 透過唯一識別碼 (科目 - 作業內容) 來找到要刪除的列
    if task_identifier in df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1).values:
        df = df[df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1) != task_identifier]
        save_data_to_sheet(df)
        # 更新下拉選單
        new_choices = get_task_choices()
        return "✅ 任務刪除成功！", df, gr.Dropdown(choices=new_choices, value=None)
    else:
        return "🛑 找不到要刪除的任務。", df, gr.Dropdown(choices=get_task_choices())


def query_tasks(status_filter, start_date, end_date):
    """根據狀態和日期範圍查詢任務"""
    df = load_data_from_sheet()
    result = df.copy() # 使用副本進行篩選

    # 1. 狀態篩選
    if status_filter != "全部":
        result = result[result["狀態"] == status_filter]

    # 2. 日期範圍篩選
    if start_date and end_date:
        result['繳交日期'] = pd.to_datetime(result['繳交日期'])
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        result = result[(result["繳交日期"] >= start) & (result["繳交日期"] <= end)]
        # 為了顯示一致，再轉回字串
        result['繳交日期'] = result['繳交日期'].dt.strftime('%Y-%m-%d')

    return result

def export_to_json(df):
    """將 DataFrame 匯出為 JSON 檔案"""
    records = df.to_dict('records')
    with open('作業紀錄.json', 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    return '作業紀錄.json'

def export_to_csv(df):
    """將 DataFrame 匯出為 CSV 檔案"""
    df.to_csv('作業紀錄.csv', index=False, encoding='utf_8_sig')
    return '作業紀錄.csv'

def get_task_choices():
    """獲取目前所有任務，用於刪除的下拉選單"""
    df = load_data_from_sheet()
    if df.empty:
        return []
    # 產生一個易於辨識的任務列表，例如："科目 - 作業內容"
    choices = df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1).tolist()
    return choices

In [5]:
# 區塊 4: Gradio UI 介面 (已修正版本)
import gradio as gr
import plotly.express as px

def create_plot(df):
    """建立狀態分佈的圓餅圖"""
    if df.empty or '狀態' not in df.columns or df['狀態'].isnull().all():
        return None # 如果沒有資料則不顯示圖表
    status_counts = df['狀態'].value_counts().reset_index()
    status_counts.columns = ['狀態', '數量']
    fig = px.pie(status_counts, names='狀態', values='數量', title='作業狀態分佈', hole=.3)
    return fig

def update_ui_components():
    """一個統一的函數來更新所有UI組件"""
    df = load_data_from_sheet()
    plot = create_plot(df)
    choices = get_task_choices()
    return df, plot, gr.Dropdown(choices=choices, value=None)

# --- Gradio 介面設計 ---
with gr.Blocks(theme=gr.themes.Soft(), title="作業紀錄提醒系統") as app:
    gr.Markdown("# ✅ 作業紀錄提醒系統")
    gr.Markdown("一個整合 Google Sheet 的作業管理工具，由 Gemini 製作。")

    with gr.Row():
        # 左側：資料顯示與視覺化
        with gr.Column(scale=2):
            gr.Label("目前作業總覽")
            output_df = gr.DataFrame(load_data_from_sheet, interactive=False)
            output_plot = gr.Plot(value=create_plot(load_data_from_sheet()))

        # 右側：功能操作區
        with gr.Column(scale=1):
            with gr.Tabs():
                # 1. 新增任務分頁
                with gr.TabItem("📝 新增任務"):
                    add_subject = gr.Textbox(label="科目")
                    add_content = gr.Textbox(label="作業內容")
                    # --- 修正點 1: 將 gr.Date 改為 gr.Textbox ---
                    add_due_date = gr.Textbox(label="繳交日期", placeholder="格式: YYYY-MM-DD")
                    add_status = gr.Radio(["未完成", "已完成", "進行中"], label="狀態", value="未完成")
                    add_button = gr.Button("新增任務", variant="primary")
                    add_status_msg = gr.Textbox(label="執行狀態", interactive=False)

                # 2. 刪除任務分頁
                with gr.TabItem("🗑️ 刪除任務"):
                    delete_dropdown = gr.Dropdown(choices=get_task_choices(), label="選擇要刪除的任務")
                    delete_button = gr.Button("確認刪除", variant="stop")
                    delete_status_msg = gr.Textbox(label="執行狀態", interactive=False)

                # 3. 查詢與匯出分頁
                with gr.TabItem("🔍 查詢與匯出"):
                    gr.Markdown("### 條件查詢")
                    query_status = gr.Dropdown(["全部", "未完成", "已完成", "進行中"], label="依狀態查詢", value="全部")
                    # --- 修正點 2 & 3: 將 gr.Date 改為 gr.Textbox ---
                    query_start_date = gr.Textbox(label="開始日期", placeholder="格式: YYYY-MM-DD")
                    query_end_date = gr.Textbox(label="結束日期", placeholder="格式: YYYY-MM-DD")
                    query_button = gr.Button("執行查詢", variant="secondary")
                    gr.Markdown("---")
                    gr.Markdown("### 匯出資料")
                    json_button = gr.Button("匯出 JSON")
                    csv_button = gr.Button("匯出 CSV")
                    download_file = gr.File(label="下載檔案")

    # --- 按鈕事件綁定 (此部分無須變更) ---
    # 新增按鈕
    add_button.click(
        fn=add_task,
        inputs=[add_subject, add_content, add_due_date, add_status],
        outputs=[add_status_msg, output_df]
    ).then(
        fn=update_ui_components,
        inputs=[],
        outputs=[output_df, output_plot, delete_dropdown] # 新增後同步更新所有組件
    )

    # 刪除按鈕
    delete_button.click(
        fn=delete_task,
        inputs=[delete_dropdown],
        outputs=[delete_status_msg, output_df, delete_dropdown]
    ).then(
        fn=update_ui_components,
        inputs=[],
        outputs=[output_df, output_plot, delete_dropdown] # 刪除後同步更新
    )

    # 查詢按鈕
    query_button.click(
        fn=query_tasks,
        inputs=[query_status, query_start_date, query_end_date],
        outputs=[output_df] # 僅更新資料表
    ).then(
        fn=lambda df: create_plot(df), # 根據查詢結果更新圖表
        inputs=[output_df],
        outputs=[output_plot]
    )

    # 匯出按鈕
    json_button.click(fn=export_to_json, inputs=[output_df], outputs=[download_file])
    csv_button.click(fn=export_to_csv, inputs=[output_df], outputs=[download_file])

# 啟動 Gradio 應用
app.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b7c3ae1ccf4d153f8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
✅ 資料已成功同步至 Google Sheet！
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b7c3ae1ccf4d153f8f.gradio.live
